In [1]:
# Load packages, utilities, dotenv
import pandas as pd
import os

from etl_utils import extract_query

from dotenv import load_dotenv
load_dotenv()

db_conn_str = f"mysql+pymysql://{os.getenv('DB_USERNAME')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/earnings"

In [27]:
# Load income statement
df = extract_query(db_conn_str, "SQL\quarterly_income_statement.sql")

Connection Successful!
SQL script at quarterly_income_statement.sql executed successfully!


In [29]:
# Balance sheet components
statement_components = ["assets", "equity", "liabilities"]
dfs = []

for component in statement_components:
    query_path = f"SQL\quarterly_balance_sheet_{component}.sql"

    res =  extract_query(db_conn_str, query_path)

    dfs.append(res)

Connection Successful!
SQL script at quarterly_balance_sheet_assets.sql executed successfully!
Connection Successful!
SQL script at quarterly_balance_sheet_equity.sql executed successfully!
Connection Successful!
SQL script at quarterly_balance_sheet_liabilities.sql executed successfully!


In [31]:
# Merge all dataframes
for df_i in dfs:
    cur_rows = len(df)

    df = df.merge(df_i, how = "left", on = ["date", "act_symbol"])

    if len(df) != cur_rows:
        print("Row number changed indicating duplicates. Ending process.")
        break

In [38]:
# Compute some static metrics
df[df.isna().any(axis = 1)]

,date,act_symbol,sales,gross_profit,net_income,diluted_net_eps,total_current_assets,total_assets,total_equity,book_value_per_share,current_portion_long_term_debt,total_current_liabilities,long_term_debt,total_liabilities
115,2021-03-31,AADI,0.000000e+00,0.000000e+00,-4440000.0,-1.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,2025-03-31,AAP,2.583000e+09,1.109000e+09,24000000.0,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,2022-06-30,AATC,2.820000e+06,2.000000e+06,70000.0,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
478,2017-12-31,AAU,0.000000e+00,0.000000e+00,-1080000.0,-0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480,2018-06-30,AAU,0.000000e+00,0.000000e+00,-910000.0,-0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165050,2022-12-31,ZTEK,1.000000e+04,-1.900000e+05,-2390000.0,-0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165053,2023-09-30,ZTEK,1.000000e+04,-1.000000e+05,-2540000.0,-0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165055,2024-03-31,ZTEK,0.000000e+00,-9.000000e+04,-1820000.0,-0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165057,2024-09-30,ZTEK,1.000000e+04,-1.600000e+05,-2230000.0,-0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
dfs[0].loc[dfs[0]['act_symbol'] == "ZTEK"]

,date,act_symbol,total_current_assets,total_assets
176171,2021-03-31,ZTEK,3000000.0,23000000.0
176172,2021-06-30,ZTEK,4000000.0,26000000.0
176173,2021-09-30,ZTEK,3000000.0,4000000.0
176174,2022-06-30,ZTEK,22000000.0,28000000.0
176175,2022-09-30,ZTEK,18000000.0,24000000.0
176176,2023-03-31,ZTEK,14000000.0,25000000.0
176177,2023-06-30,ZTEK,11000000.0,23000000.0
176178,2023-12-31,ZTEK,7000000.0,18000000.0
176179,2024-06-30,ZTEK,3000000.0,15000000.0
176180,2024-12-31,ZTEK,3000000.0,14000000.0
